PROJET : TOPIC MODELING

Dépendences

In [2]:
# # In case of import errors
# !pip install nltk
# !pip install textblob
import os
import numpy as np
import pandas as pd
import re
import nltk
import string
from nltk.corpus import stopwords
import re
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

# # In case of any corpus are missing 
# download all-nltk
#nltk.download()
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from PIL import Image

PARTIE 1 : WORDCLOUD

In [215]:
#Récupère le texte d'un livre sans les mentions du projet Gutenberg
def get_text_book(gutenberg_book):
    #suppression de l'entête
    if gutenberg_book.find("*** START")!= -1:
        end_head = gutenberg_book.find("*** START")
    elif  gutenberg_book.find("***START")!= -1: 
        end_head = gutenberg_book.find("***START")
    else:
        end_head = 1
    book = gutenberg_book.replace(gutenberg_book[:end_head], "")
    #Suppression du footer
    if book.find("*** END")!= -1:
        start_footer = book.find("*** END")
    elif book.find("***END")!= -1:
        start_footer = book.find("***END")
    else:
        start_footer = (len(book)-1)
        #print("start_footer else: " , start_footer)
    book = book.replace(book[start_footer:], "")
    return book

In [81]:
# operations de Cleaning 
def get_tokenized_book(book):
    #passe tout en minuscule
    book=book.lower()
    #Supprime la ponctuation
    translator = str.maketrans('', '', string.punctuation)
    book = book.translate(translator)
    #Supprime les espaces
    book = " ".join(book.split())
    #suppression de tout ce qui n'est pas des lettres
    book = re.sub('[^A-Za-z]', ' ', book)
    #Supprime les mots vides
    stop_words = set(stopwords.words("english"))
    stop_words.add("gutenberg")
    stop_words.add("gutenbergtm")
    stop_words.add("project")
    stop_words.add("ebook")
    #print(stop_words)
        #Separe les mots 
    tokenized_book = word_tokenize(book)
    filtered_text = [word for word in tokenized_book if word not in stop_words]
    #Trouve la racine des mots
    stemmer = PorterStemmer()
    stems = [stemmer.stem(word) for word in filtered_text]
    #Verifie que la racine des mots appartient à la langue 
    lemmatizer = WordNetLemmatizer()
    lemmas = [lemmatizer.lemmatize(word, pos ='v') for word in stems]
    cleaned_book=[word for word in lemmas if len(word)> 3]
    return cleaned_book


In [5]:
#création wordcloud à partir d'un book nettoyé
def get_wordcloud_by_txt(book_cleaned):
    wordcloud = WordCloud(width = 1800,
                          height = 1800,
                          background_color = 'white',
                          stopwords = stop_words,
                          min_font_size = 10,
                          max_words = 50).generate(book_cleaned)
    return wordcloud

# plt.imshow(wordcloud)
# plt.axis("off")
# plt.show();

In [84]:
#retourne un bag of words à partir d'un livre tokenisé
def get_bag_of_words(tokenized_book):
    words_dict={}
    text_book = " ".join(tokenized_book)
    for word in tokenized_book:
        freq_word= text_book.count(word)/len(tokenized_book)
        words_dict[word]=freq_word
    bag_of_words = pd.DataFrame([words_dict], columns=words_dict.keys())
    return bag_of_words.iloc[0].sort_values(ascending = False)[:1000]

In [83]:
def get_wordcloud_by_bow(bag_of_words):
    wordcloud = WordCloud(min_word_length = 3,background_color='white', max_words = 50)
    wordcloud.generate_from_frequencies(bag_of_words)
    return wordcloud

In [85]:
#récupère la liste des 52 livres
book_names = os.listdir('./data_books')
#print(books)

books_cleaned=[]

#boucle sur la liste des livres
for book_name in book_names:
    f = open("data_books/" + book_name, encoding='UTF-8' )
    
    #Récupère le texte d'un livre sans les mentions du projet Gutenberg
    book=get_text_book(f.read())
    
    #Nettoyage et tokenization des textes
    tokenized_book = get_tokenized_book(book)
    
    #Enregistrement du livre nettoyé
    book_cleaned = " ".join(tokenized_book)
    books_cleaned.append(book_cleaned)
    #Sauvegarde des books_cleaned
    svg_book=open("books_cleaned/"+book_name,"w",encoding="utf-8")
    svg_book.write(book_cleaned)
    svg_book.close()
    
    #création des wordclouds à partir du text nettoyé
    #wordcloud = get_wordcloud_by_txt(book_cleaned)
    #wordcloud.to_file("img/"+ book_name[0:-4] +".png")
    
    #création des bags_of_words
    bag_of_words = get_bag_of_words(tokenized_book)
    bag_of_words.to_csv("bags_of_words/"+ book_name[0:-4] +".csv")
    
    #Création du wordcloud à partir du bag of words
    wordcloud = get_wordcloud_by_bow(bag_of_words)
    wordcloud.to_file("wordclouds/"+ book_name[0:-4] +".png")
    
    f.close()
    
print("Nombre books_cleaned enregistrés: " ,len(books_cleaned))

Nombre books_cleaned enregistrés:  52


PARTIE 2 : TOPIC MODELING

In [26]:
#récupère la liste des 52 livres
import os
book_names = os.listdir('./data_books')
#print(books)

books_cleaned=[]

#boucle sur la liste des livres
for book_name in book_names:
    #if os.path.isdir("books_cleaned/"+book_name)== False:
    f = open("data_books/" + book_name, encoding='UTF-8' )

    #Récupère le texte d'un livre sans les mentions du projet Gutenberg
    book=get_text_book(f.read())

    #Nettoyage et tokenization des textes
    tokenized_book = get_tokenized_book(book)

    #Enregistrement du livre nettoyé
    book_cleaned = " ".join(tokenized_book)
    books_cleaned.append(book_cleaned)

    #print("Nombre books_cleaned enregistrés : " ,len(books_cleaned))
    f.close()
    
print("Nombre books_cleaned enregistrés final: " ,len(books_cleaned))
    


Nombre books_cleaned enregistrés final:  52


---------- Paramètres d'analyse

In [127]:
#nbr de caractéristique = nbr de mots
n_features = 1000
#nbr de topic /sujet à extraire
n_components = 3
#Etiquettes des topics pour model LSA
lsa_topics = ["Topic 1", "Topic 2", "Topic 3"]
#nbr de mots représentatifs sélectionnés pour l'analyse
n_top_words = 10

In [128]:
#Création de la Matrix TF-IDF    
from sklearn.feature_extraction.text import TfidfVectorizer 

# Configuration d'un TF-IDF Vectorizer 
#supprime les mots qui apparaissent dans moins de 2 livres ou dans au moins 95% des livres
#Le réglage ngram_rangesur (1, 2)indique que nous utilisons uni-gram et bi-gram lors de la vectorisation
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2,
                                   max_features = n_features,
                                   stop_words = 'english',
                                   ngram_range=(1, 2),
                                   use_idf = True) 
 
#Création de la matrix  
tfidf_vectors = tfidf_vectorizer.fit_transform(books_cleaned)

In [ ]:
#création d'un dataframe pour afficher la Matrix
document_term_matrix=pd.DataFrame(index = tfidf_vectorizer.get_feature_names())
i=0
for vector in tfidf_vectors:
    document_term_matrix.insert( i , book_names[i] , vector.T.todense(), allow_duplicates=False)
    i += 1
document_term_matrix

PARTIE 3 : LATENTE SEMENTIC ANALYSIS (L.S.A)

In [110]:
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import random as sparse_random

#Instanciation d'un analyseur LSA
#random_state=1 permet d'ajuster l'instance une fois avec des valeurs prédéfinies
#pour éviter le problème de sign indeterminnancy
lsa = TruncatedSVD(n_components = n_components, random_state = 1, algorithm = 'arpack')

#Ajustement / entrainement avec la matrix TF-IDF_vectors
lsa.fit(tfidf_vectors)

TruncatedSVD(algorithm='arpack', n_components=3, random_state=1)

In [125]:
#Retourne les topics d'après le model LSA
def get_lsa_topics(lsa, vectorizer, topics, n_top_words = n_top_words):
    word_dict = {}
    #Récupère la liste des n mots de la Matrix
    feature_names = vectorizer.get_feature_names()
    #Boucle sur les n topics
    for topic_idx, topic in enumerate(lsa.components_):
        #Récupère la listes des indices des n mots ayant les meilleurs values pour le topic concerné
        top_features_ind = topic.argsort()[:-n_top_words - 1:-1]
        #Récupère la listes des n mots grâce à leur indice
        top_features = [feature_names[i] for i in top_features_ind]
        #Enregistre la liste des meilleurs mots pour chaque topic avec comme clef l'étiquette du topic
        word_dict[topics[topic_idx]] = top_features

    return pd.DataFrame(word_dict)

In [210]:
#nbr de mots représentatifs sélectionnés pour l'analyse
n_top_words = 10
top_words_by_topic = get_lsa_topics(lsa, tfidf_vectorizer, lsa_topics)
top_words_by_topic


,Topic 1,Topic 2,Topic 3
0,littl,acid,acid
1,time,state,water
2,think,nation,solut
3,like,unit state,carbon
4,look,form,heat
5,great,term,soap
6,king,unit,substanc
7,tell,copi,quantiti
8,good,water,boil
9,thing,distribut,food


In [132]:
#Retourne le topic du texte passé en paramètre
def get_inference(model, vectorizer, topics, text, threshold):
    v_text = vectorizer.transform([text])
    scores = model.transform(v_text)

    labels = set()
    for i in range(len(scores[0])):
        if scores[0][i] > threshold:
            labels.add(topics[i])

    if not labels:
        return 'None', -1, set()

    return topics[np.argmax(scores)], scores, labels

In [212]:
#ANALYSE LSA

#Récupèration des textes des livres nettoyés
book_names = os.listdir('./books_cleaned')

books_scores_dict={}

#Déclaration des listes de livres par topic
books_topic1=[]
books_topic2=[]
books_topic3=[]

#boucle sur la liste des livres
for i,book_name in enumerate(book_names):
    #Récupération des résultats prédis pour chaque livre
    book = open("books_cleaned/" + book_name, encoding='UTF-8' ).read()
    topic_predicted, scores, _ = get_inference(lsa, tfidf_vectorizer, lsa_topics, book, 0)
    
    #Enregistrement des scores de chaque livre
    books_scores_dict[i+1]= scores[0]
    
    #Répartition des livres dans chaque topic
    if topic_predicted == "Topic 1":
        books_topic1.append(book_name)
    elif topic_predicted == "Topic 2":
        books_topic2.append(book_name)
    elif topic_predicted == "Topic 3":
        books_topic3.append(book_name)
    else:
        print(book_name, "n'a pas été classé!")

#Création Dataframe reprenant les scores de chaque livre par Topic
books_scores_by_topic = pd.DataFrame(books_scores_dict, index = lsa_topics).T
books_scores_by_topic["Books Names"]= book_names


#Création des listes de livres classés par Topic
list_books_topic1=pd.DataFrame(books_topic1, columns=["Topic 1"])
list_books_topic2=pd.DataFrame(books_topic2, columns=["Topic 2"])
list_books_topic3=pd.DataFrame(books_topic3, columns=["Topic 3"])


In [214]:
#Affichages

#books_scores_by_topic.reindex(columns = ["Books Names", "Topic 1", "Topic 2", "Topic 3"])

list_books_topic1

# list_books_topic2

# list_books_topic3


,Topic 1
0,alice-in-wonderland.txt
1,Among the Forest People.txt
2,Curious Myths of the Middle Ages.txt
3,"Democracy In America, Volume 1 (of 2).txt"
4,Histories of two hundred and fifty-one divisio...
5,History of King Charles The First of England.txt
6,How the Flag Became Old Glory.txt
7,Little Lord Fauntleroy.txt
8,Medieval People.txt
9,Mother Storie.txt


PARTIE 4 : DOCUMENT SIMILARITY

In [ ]:
#
def best_recommended_books(book_name, nbr_book_reco):
    list_recommended_book = []
    
    return list_recommended_book

In [ ]:
#
nbr_book_reco = 5
book_name = "alice-in-wonderland"

list_recommended_book = best_recommended_books(book_name, nbr_book_reco)
print(list_recommended_book)